In [1]:
# imports

# langchain
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.embeddings import GPT4AllEmbeddings
from langchain.evaluation.qa import QAGenerateChain         # use in our paper
from langchain.vectorstores import DocArrayInMemorySearch

# langchain-ollama
from langchain_ollama import ChatOllama

/Users/cmoyacal/miniforge3/envs/langchain/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
# upload the doc
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [3]:
# define embeddings for doc and query
embeddings = GPT4AllEmbeddings()

In [4]:
db = DocArrayInMemorySearch.from_documents(
    data, 
    embeddings
)

/Users/cmoyacal/miniforge3/envs/langchain/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
# define llm
llm = ChatOllama(
    model="llama3.1",
    temperature=0.0,
)


# define retrienverQA
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=db.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [6]:
# test data points
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [7]:
data[11]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

In [8]:
# hard coded examples
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [9]:
# LLM generated example  - this for our paper
example_gen_chain = QAGenerateChain.from_llm(ChatOllama(model="llama3.1"))

In [10]:
# we need this - generate examples
new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in data[:3]])

/Users/cmoyacal/miniforge3/envs/langchain/lib/python3.12/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [11]:
new_examples[0]

{'qa_pairs': {'query': "What type of material is used to make the insoles of the Women's Campside Oxfords?",
  'answer': 'EVA (Comfortable EVA innersole with Cleansport NXT antimicrobial odor control)'}}

In [12]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [13]:
# combine examples
examples += new_examples
print(examples)

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}, {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}, {'qa_pairs': {'query': "What type of material is used to make the insoles of the Women's Campside Oxfords?", 'answer': 'EVA (Comfortable EVA innersole with Cleansport NXT antimicrobial odor control)'}}, {'qa_pairs': {'query': 'What percentage of materials used in the construction of the Recycled Waterhog dog mat is recycled?', 'answer': '94%'}}, {'qa_pairs': {'query': "What percentage of the sun's harmful rays is blocked by the UPF 50+ rated fabric in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?", 'answer': '98%'}}]


In [14]:
# get answers from LLM
qa.run(examples[0]["query"])




> Entering new RetrievalQA chain...


/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_53803/2925013409.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])



> Finished chain.


'Yes, according to the product description, the Cozy Comfort Pullover Set (Stripe) has pull-on pants with a wide elastic waistband and drawstring, side pockets, and a modern slim leg.'

In [15]:
# manual evaluation
import langchain
langchain.debug = True

In [16]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

'Yes, according to the product description, the Cozy Comfort Pullover Set (Stripe) has pull-on pants with a wide elastic waistband and drawstring, side pockets, and a modern slim leg.'

In [17]:
# Turn off the debug mode
langchain.debug = False

In [18]:
# assisted evaluation
predictions = qa.apply(examples[:2])



> Entering new RetrievalQA chain...


/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_53803/1986035667.py:2: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  predictions = qa.apply(examples[:2])



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [19]:
from langchain.evaluation.qa import QAEvalChain

In [20]:
llm = ChatOllama(temperature=0, model="llama3.1")
eval_chain = QAEvalChain.from_llm(llm)